In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

# Load dataset
df = pd.read_csv("/kaggle/input/datas/Crop_recommendation.csv")

In [1]:
# Separate features and labels
X_num = df[["temperature", "humidity", "ph", "water availability"]]

# Encode season
season_enc = OneHotEncoder(sparse=False)
X_season = season_enc.fit_transform(df[["season"]])

# Combine numeric + season
import numpy as np
X = np.hstack([X_num.values, X_season])

# Encode labels
label_enc = LabelEncoder()
y = label_enc.fit_transform(df["label"])

# One-hot target
y = tf.keras.utils.to_categorical(y)

# Normalize numeric features
scaler = StandardScaler()
X[:, :4] = scaler.fit_transform(X[:, :4])

NameError: name 'df' is not defined

In [2]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation="relu", input_shape=(X.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(y.shape[1], activation="softmax")
])

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Train
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=16)

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")

# Example prediction
sample = np.array([[28, 80, 6.5, 300]])  # numeric values
sample_season = season_enc.transform([["rainy"]])
sample_input = np.hstack([sample, sample_season])
sample_input[:, :4] = scaler.transform(sample_input[:, :4])

pred = model.predict(sample_input)
print("Predicted Crop:", label_enc.inverse_transform([np.argmax(pred)]))

NameError: name 'train_test_split' is not defined

In [ ]:
price = pd.read_csv("/kaggle/input/data-gov-in/9ef84268-d588-465a-a308-a864a43d0070.csv")
df_filtered = price[price["State"].str.lower() == "Odisha"]

In [ ]:
df_filtered

In [ ]:
import requests
import pandas as pd

# Example: Raipur, Chhattisgarh
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 21.25,
    "longitude": 81.63,
    "hourly": "temperature_2m,relative_humidity_2m",
    "timezone": "auto"
}

resp = requests.get(url, params=params).json()

# Convert to DataFrame
hourly = pd.DataFrame({
    "date": pd.to_datetime(resp["hourly"]["time"]),
    "temperature_2m": resp["hourly"]["temperature_2m"],
    "humidity_2m": resp["hourly"]["relative_humidity_2m"]
})

# Derive season from month
def get_season(month):
    if month in [12, 1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "summer"
    elif month in [6, 7, 8, 9]:
        return "rainy"
    else:
        return "spring"

hourly["season"] = hourly["date"].dt.month.map(get_season)

print(hourly.head())

In [ ]:
import requests
import pandas as pd

# Raipur, Chhattisgarh
lat, lon = 21.25, 81.63

url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": lat,
    "longitude": lon,
    "current": "temperature_2m,relative_humidity_2m,apparent_temperature,is_day,precipitation,"
               "rain,showers,snowfall,weather_code,cloudcover,pressure_msl,surface_pressure,"
               "windspeed_10m,winddirection_10m,windgusts_10m",
    "timezone": "auto",
    "daily":"rain_sum"
}

resp = requests.get(url, params=params).json()

# Current weather dict
current_weather = resp["current"]

# Convert to DataFrame (1-row table)
df = pd.DataFrame([current_weather])
print(df.T)  # transpose for easier view

In [ ]:
import requests

def get_soil_ph(lat, lon, stat="mean"):
    """
    Query SoilGrids REST API for soil pH in water (phh2o) at a location.
    Returns dict: { "0-5cm": 8.2, "5-15cm": 8.1, ... }
    """
    url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    params = {"lat": lat, "lon": lon, "property": "phh2o"}
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()

    out = {}
    try:
        layers = data["properties"]["layers"]
        ph_layer = next(layer for layer in layers if layer["name"] == "phh2o")
        for d in ph_layer["depths"]:
            label = d["label"]  # e.g. "0-5cm"
            val = d["values"].get(stat)
            if val is not None:
                out[label] = val / 10.0  # convert to real pH
    except Exception as e:
        print("Parse error:", e)
    return out


if __name__ == "__main__":
    lat, lon = 21.25, 81.63
    ph_profile = get_soil_ph(lat, lon)
    if ph_profile:
        print("Soil pH profile:")
        for depth, ph in ph_profile.items():
            print(f"  {depth}: {ph:.2f}")
    else:
        print("No pH data found.")

In [ ]:
import requests
from datetime import date, timedelta

def get_total_rainfall(latitude, longitude):
    # Last 30 days
    end_date = date.today()
    start_date = end_date - timedelta(days=30)

    url = (
        f"https://archive-api.open-meteo.com/v1/archive"
        f"?latitude={latitude}&longitude={longitude}"
        f"&start_date={start_date}&end_date={end_date}"
        f"&daily=precipitation_sum&timezone=auto"
    )

    response = requests.get(url)
    data = response.json()

    # Extract rainfall values, ignoring None
    rainfall_values = [
        v for v in data.get("daily", {}).get("precipitation_sum", []) if v is not None
    ]
    total_rainfall = sum(rainfall_values)

    return {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": str(start_date),
        "end_date": str(end_date),
        "daily_rainfall": rainfall_values,
        "total_rainfall_mm": total_rainfall,
        "api_url": url,
    }

# Example: Chandigarh, India (30.7333, 76.7794)
rainfall_report = get_total_rainfall(30.7333, 76.7794)
rainfall_report

In [ ]:
import requests
import pandas as pd
from datetime import date, timedelta

# ---------- 1. Weather Data ----------
def get_weather_summary(lat, lon):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "temperature_2m,relative_humidity_2m",
        "timezone": "auto"
    }
    resp = requests.get(url, params=params).json()

    df = pd.DataFrame({
        "date": pd.to_datetime(resp["hourly"]["time"]),
        "temperature_2m": resp["hourly"]["temperature_2m"],
        "humidity_2m": resp["hourly"]["relative_humidity_2m"]
    })

    avg_temp = df["temperature_2m"].mean()
    avg_hum = df["humidity_2m"].mean()

    # Season from current date
    month = date.today().month
    if month in [12, 1, 2]:
        season = "winter"
    elif month in [3, 4, 5]:
        season = "summer"
    elif month in [6, 7, 8, 9]:
        season = "rainy"
    else:
        season = "spring"

    return avg_temp, avg_hum, season


# ---------- 2. Soil pH (first depth only) ----------
def get_soil_ph_first(lat, lon, stat="mean"):
    url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    params = {"lat": lat, "lon": lon, "property": "phh2o"}
    resp = requests.get(url, params=params).json()

    try:
        layers = resp["properties"]["layers"]
        ph_layer = next(layer for layer in layers if layer["name"] == "phh2o")
        first_depth = ph_layer["depths"][0]  # first depth
        val = first_depth["values"].get(stat)
        if val is not None:
            return val / 10.0
    except Exception:
        pass

    return None


# ---------- 3. Rainfall ----------
def get_total_rainfall(lat, lon):
    end_date = date.today()
    start_date = end_date - timedelta(days=30)
    url = (
        f"https://archive-api.open-meteo.com/v1/archive"
        f"?latitude={lat}&longitude={lon}"
        f"&start_date={start_date}&end_date={end_date}"
        f"&daily=precipitation_sum&timezone=auto"
    )
    resp = requests.get(url).json()
    rainfall_values = [v for v in resp.get("daily", {}).get("precipitation_sum", []) if v is not None]
    return sum(rainfall_values)


# ---------- 4. Combine into one row ----------
def build_single_row(lat, lon, label="sample"):
    avg_temp, avg_hum, season = get_weather_summary(lat, lon)
    soil_ph = get_soil_ph_first(lat, lon)
    rainfall = get_total_rainfall(lat, lon)

    df = pd.DataFrame([{
        "temperature": avg_temp,
        "humidity": avg_hum,
        "ph": soil_ph,
        "water availability": rainfall,
        "season": season,
    }])
    return df


# Example: Raipur, Chhattisgarh
lat, lon =  26.0909,  81.7131
dataset = build_single_row(lat, lon, label="raipur_sample")

print(dataset)